<a href="https://colab.research.google.com/github/shayesteh99/Tehran-Traffic/blob/main/find_neighbors_in_graph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install igraph

     |████████████████████████████████| 3.2 MB 4.3 MB/s 


In [ ]:
import pandas as pd
import igraph
from igraph import *
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
%cd 'gdrive/MyDrive/Neshan2'
%ls

/content/gdrive/.shortcut-targets-by-id/1kb7msLseHDgbNcoeHcAexfHINwu2zRER/Neshan2
check_neighbors_average.ipynb
Classification/
DATA/
find_common_days.ipynb
find_good_segments.ipynb
find_mapped_good_neighors.ipynb
find_neighbors.ipynb
find_shorter_neighbors.ipynb
get_important_good_segment_information.ipynb
get_mapped_good_segment_information.ipynb
GOOD_DATA/
GOOD_IMPORTANT_DATA/
good_important_segment_information.csv
GOOD_MAPPED_DATA/
good_mapped_segment_information.csv
good_segment_information.csv
Important_segment.csv
just_good_neighbors_1000.csv
just_good_neighbors_1500.csv
make_mapped_good_days.ipynb
MAPPED/
mapped_good_data.csv
mapped_good_igraph.ipynb
mapped_good_neighors.csv
mapped_important_good_neighors.csv
mapped_segment.csv
NEIGHBOR/
neighbors_1000.csv
neighbors_750.csv
new_data_df.csv
PP/
Regression/
segment1.txt
segment_locatiion.ipynb
segments_corrected.csv
Shayesteh/
To_verify.ipynb
to_verify_mlpregressor.ipynb
variance.ipynb
vars.txt


In [ ]:
segment_info = pd.read_csv("good_important_segment_information.csv")
print(segment_info.columns)

Index(['Unnamed: 0', 'name', 'segment_id', 'Latitude', 'Longitude'], dtype='object')


In [ ]:
df = pd.read_csv('new_data_df.csv')
df_graph = df[['from_node', 'to_node']]
raw_edges = df_graph.values

In [ ]:
g = Graph(directed = True)
edges = []
for r in raw_edges:
  edges += [(r[0], r[1])]
vertices = list(set(raw_edges.flatten()))
g.add_vertices(max(vertices) + 1)
g.add_edges(edges)

In [ ]:
print(vertices)

[524289, 1, 262148, 8, 9, 524300, 524301, 262158, 15, 524304, 262163, 21, 524315, 28, 524319, 262176, 262177, 524326, 40, 41, 43, 262187, 45, 524334, 262190, 262191, 49, 50, 262193, 262196, 53, 54, 262199, 524344, 262200, 61, 262206, 63, 524351, 262210, 524355, 73, 524365, 262222, 81, 83, 524372, 84, 86, 87, 262227, 262228, 93, 524382, 94, 95, 262241, 524387, 262245, 262246, 102, 524392, 262249, 101, 524395, 262250, 110, 524399, 112, 113, 262260, 524405, 262261, 116, 120, 524409, 524411, 262267, 524414, 262271, 127, 129, 262276, 132, 524423, 524424, 262281, 136, 524428, 141, 144, 262289, 262290, 524434, 150, 151, 152, 524441, 262298, 524442, 262299, 153, 262300, 262303, 161, 524454, 262312, 524461, 174, 176, 524465, 262320, 262321, 181, 182, 262329, 187, 524478, 262334, 262335, 524483, 200, 524489, 262346, 524491, 262347, 202, 204, 262344, 208, 210, 524499, 211, 1310933, 213, 524502, 212, 262357, 218, 524507, 262364, 1048791, 262367, 1310944, 524512, 1048801, 524515, 262373, 1310950, 2

In [ ]:
def find_k_neighbors(graph, vertex, k, mode):
  neis = []
  dist = {}
  dist[vertex] = 0
  bfs = [vertex]
  while len(neis) < k:
    if len(bfs) < 1:
      break
    v = bfs.pop(0)
    n = graph.neighbors(v, mode=mode)
    for i in n:
      if i not in neis and i != vertex:
        dist[i] = dist[v] + 1
        bfs.append(i)
        neis.append(i)
  neis = neis[:k]
  return neis, dist

In [ ]:
k = 3
neighbors = []
in_len = []
out_len = []
not_enough_neis = 0
not_valid_ids = 0
no_seg_ids = 0
expected_len = 1 + 2 * 3 * k
for i in range(len(df)):
  row = df.iloc[i].to_numpy()
  seg_id = row[1]
  begin = row[2]
  end = row[3]

  in_neis, in_dist = find_k_neighbors(g, begin, k, "in")
  out_neis, out_dist = find_k_neighbors(g, end, k, "out")
  if len(in_neis) < k or len(out_neis) < k:
    not_enough_neis += 1
    continue


  sample = [seg_id]
  inf = segment_info[segment_info['segment_id'] == seg_id].to_numpy()
  if len(inf) == 0:
    not_valid_ids += 1
    continue
  lat, lon = inf[0][3], inf[0][4]

  for n in in_neis:
    nei_inf = df[df['to_node'] == n].to_numpy()
    if len(nei_inf) == 0:
      no_seg_ids += 1
      break
    seg = nei_inf[0][1]
    seg_inf = segment_info[segment_info['segment_id'] == seg].to_numpy()
    if len(seg_inf) == 0:
      not_valid_ids += 1
      break
    dist = math.sqrt((seg_inf[0][3] - lat)**2 + (seg_inf[0][4] - lon)**2)
    sample.append(seg)
    sample.append(-1 * in_dist[n])
    sample.append(dist)

  for n in out_neis:
    nei_inf = df[df['from_node'] == n].to_numpy()
    if len(nei_inf) == 0:
      no_seg_ids += 1
      break
    seg = nei_inf[0][1]
    seg_inf = segment_info[segment_info['segment_id'] == seg].to_numpy()
    if len(seg_inf) == 0:
      not_valid_ids += 1
      break
    dist = math.sqrt((seg_inf[0][3] - lat)**2 + (seg_inf[0][4] - lon)**2)
    sample.append(seg)
    sample.append(out_dist[n])
    sample.append(dist)
  if len(sample) == expected_len:
    neighbors.append(sample)

In [ ]:
print(len(neighbors))

6683


In [ ]:
columns = ['seg_id']
for i in range(k):
  columns.append('in_id' + str(i+1))
  columns.append('in_graph_dist' + str(i+1))
  columns.append('in_uc_dist' + str(i+1))
for i in range(k):
  columns.append('out_id' + str(i+1))
  columns.append('out_graph_dist' + str(i+1))
  columns.append('out_uc_dist' + str(i+1))
neighbors_df = pd.DataFrame(neighbors, columns = columns)
neighbors_df.to_csv("Shayesteh/neighbors/graph_neighbors_" + str(k) + ".csv")